# Ants Bees VGG16 による解析

# Fitting 

# TypeBによるTrainingクラスの作成

In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
'''
3. 前処理前後の画像を確認する
'''
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
import torch.utils.data as data
from pathlib import Path


In [ ]:
data_dir = '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data'
root_dir= '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data'

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AntsBees')

###Deviceの設定

In [ ]:
# 使用可能なデバイス(CPUまたはGPU）を取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
torch.cuda.synchronize()
print(torch.cuda.memory_allocated())

0


#(1) DataSet の作成

### train_list, val_list の作成

In [ ]:
import glob
import pprint

def make_datapath_list(phase='train'):
    '''
    データのファイルパスを格納したリストを作成する。

    Parameters:
      phase(str): 'train'または'val'

    Returns:
      path_list(list): 画像データのパスを格納したリスト
    '''
    # 画像ファイルのルートディレクトリ
    # 画像ファイルパスのフォーマットを作成
    # rootpath +
    #   train/ants/*.jpg
    #   train/bees/*.jpg
    #   val/ants/*.jpg
    #   val/bees/*.jpg
    tt= phase +'/**/*.jpg'
    target_path = os.path.join(root_dir,tt)
    # ファイルパスを格納するリスト
    path_list = []  # ここに格納する

    # glob()でファイルパスを取得してリストに追加
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list

# ファイルパスのリストを生成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

print(train_list)

# 訓練データのファイルパスの前後5要素ずつ出力
print('train')
pprint.pprint(train_list[:5])
pprint.pprint(train_list[-6:-1])
# 検証データのファイルパスの前後5要素ずつ出力
print('val')
pprint.pprint(val_list[:5])
pprint.pprint(val_list[-6:-1])

['/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/0013035.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1095476100_3906d8afde.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1099452230_d1949d3250.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/116570827_e9c126745d.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1225872729_6f0856588f.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1262877379_64fcada201.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1269756697_0bce92cdab.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1286984635_5119e80de1.jpg', '/cont

In [ ]:
p=Path(train_list[5])
print(p.parts[-2])

p2=Path(val_list[-4])
print(p2.parts[-2])

ants
bees


#（２）DataSetの作成

In [ ]:
from dsets.dsets import MakeDataset
from util.ImageTransform import ImageTransform

In [ ]:
'''
6. データローダーの生成
'''
import torch

# ミニバッチのサイズを指定
batch_size = 10
SIZE = 224
# 標準化する際の各RGBの平均値
MEAN = (0.485, 0.456, 0.406) # ImageNetデータセットの平均値を使用
# 標準化する際の各RGBの標準偏差
STD = (0.229, 0.224, 0.225)  # ImageNetデータセットの標準偏差を使用


# 画像のサイズ、平均値、標準偏差の定数値
size, mean, std = SIZE, MEAN, STD

# MakeDatasetで前処理後の訓練データと正解ラベルを取得
train_dataset = MakeDataset(
    file_list=train_list, # 訓練データのファイルパス
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='train')
# MakeDatasetで前処理後の検証データと正解ラベルを取得
val_dataset = MakeDataset(
    file_list=val_list, # 検証データのファイルパス
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='val')


#(2) DataLoader の作成

In [ ]:
'''
6. データローダーの生成
'''
from torch.utils.data import DataLoader

# 訓練用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 検証用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



#(4) モデルの作成

In [ ]:
'''
7. 学習済みのVGG16モデルをロード
'''
from torchvision import models
import torch.nn as nn

# ImageNetで事前トレーニングされたVGG16モデルを取得
model = models.vgg16(pretrained=True)

# VGG16の出力層のユニット数を2にする
model.classifier[6] = nn.Linear(
    in_features=4096, # 入力サイズはデフォルトの4096
    out_features=2)   # 出力はデフォルトの1000から2に変更


model = model.to(device)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# For Loop with DataLoaderの研究

In [ ]:
batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)


input_t, label_t = batch_tup

input_g=input_t.to(device)
label_g=label_t.to(device)
model = model.to(device)

outputs = model(input_g)




In [ ]:
softmax = nn.Softmax(dim=1)
prob_g=softmax(outputs)
print(prob_g[:,1])

tensor([0.4320, 0.4978, 0.4076, 0.5295, 0.2694, 0.5385, 0.8632, 0.1535, 0.6495,
        0.3185], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)
print(loss_g)

tensor([0.5656, 0.6887, 0.5236, 0.6359, 1.3117, 0.7733, 1.9894, 1.8743, 0.4315,
        0.3835], device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
print(outputs)

tensor([[ 0.3076,  0.0337],
        [-0.9031, -0.9121],
        [-0.2140, -0.5879],
        [-0.2444, -0.1264],
        [ 0.8584, -0.1394],
        [-0.7858, -0.6315],
        [-1.2954,  0.5469],
        [ 0.7924, -0.9153],
        [-0.0214,  0.5956],
        [-0.4270, -1.1875]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:

batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

metrics_g= torch.zeros(3,len(train_dl.dataset),device=device)

input_t, label_t = batch_tup

input_g = input_t.to('cuda')
label_g = label_t.to('cuda')

outputs  = model(input_g)

softmax = nn.Softmax(dim=1)
prob_g= softmax(outputs)
prob_g2=prob_g[:,1]

loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)

loss_g=loss_g

print(outputs.shape)
print(prob_g2.shape)
print(label_g.shape)
print(loss_g.shape)

print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

start_ndx = 0 * 10
end_ndx = start_ndx + 10

#metrics_g[0, start_ndx:end_ndx] = label_g
#metrics_g[1, start_ndx:end_ndx] = prob_g2.detach(
#metrics_g[2, start_ndx:end_ndx] = loss_g.detach()

print(label_g)
print(prob_g2)
print(loss_g)

import sys
var_name = 'prob_g2'
print(sys.getsizeof(eval(var_name)))

torch.Size([10, 2])
torch.Size([10])
torch.Size([10])
torch.Size([10])
Allocated: 1.9 GB
Cached:    2.2 GB
tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 0], device='cuda:0')
tensor([0.2945, 0.4173, 0.6076, 0.6697, 0.6287, 0.1728, 0.3156, 0.3628, 0.7078,
        0.5649], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([1.2226, 0.5401, 0.9353, 1.1077, 0.9907, 1.7553, 1.1534, 1.0138, 1.2305,
        0.8321], device='cuda:0', grad_fn=<NllLossBackward0>)
88


#END